<a href="https://colab.research.google.com/github/bbberylll/ESAA_OB/blob/main/ESAA_OB_M05_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 텍스트 분류 실습
--> 희소 행렬 + 분류 : 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈가 적절함

* 텍스트 정규화 -> 피처 벡터화 -> 머신러닝 알고리즘 적용

  **최적의 ML 알고리즘을 선택하는 것 & 최상의 피처 전처리를 수행하는 것이 텍스트 분석에서 성능을 향상시키는 중요 요인**

### 텍스트 정규화
* 내용을 제외한 제목, 소속, 이메일 주소 등의 정보는 뉴스그룹 분류의 target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많음. --> **제거할 것!**


---


### 피처 벡터화 & 학습
countvectorizer 이용
**테스트 데이터에서 피처 벡터화를 할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 transform 해야 한다**
**test data에서는 fit_transform을 적용하면 안된다..!!**

* Count와 TF-IDF를 비교했을 때 일반적으로 후자가 더 성능이 좋다.
* grid search를 통해 최적의 파라미터 찾으면 성능 향상 가능



### 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합
수행 시간 절약 가능한 스트림 기반 처리
--> (텍스트 기반의 피처 벡터화, 모든 전처리 작업=스케일링, 벡터 정규화, PCA) + Estimator = PipeLine

텍스트 분석에서 pipeline을 이용하면 fit, transform, fit, predict 과정을 굳이 거치지 않고 단순히 마지막 fit, predict을 통해 수행되는 것을 알 수 있음

만약 GridSearchCV+Pipeline을 원한다면
--> *GridSearchCV 하이퍼 파라미터로 param_grid의 형태가 달라지게 됨*
> 객체 변수명__하이퍼 파라미터명 형태로 적어줘야 함

# 감성분석
텍스트가 나타내는 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산

### 종류
1) 지도학습 : 학습데이터와 타깃 레이블을 기반으로 감성 분석 수행 --> 다른 데이터의 감성 분석을 예측.
2) 비지도학습 : Lexicon 감성 어휘 사전 이용

### Lexicon  _ NLTK 기반
: 감성 어휘 사전. 긍정, 부정 감성 정도를 의미하는 '감성지수'를 가지고 있음
>> 단어의 위치, 주변 단어, 문맥, POS(part of speech)를 참고해 결정됨

  1. WordNet -- NLTK 패키지
  : Semantic 분석을 제공하는 어휘 사전. 같은 어휘여도 다르게 사용되는 어휘 정보를 제공함. 각각의 품사로 구성된 개별 단어를 **Synset**이라고 표현함

  2. SentiWordNet : sentiment score를 할당(긍정, 부정 감성 지수 + 객관성 지수)
  3. Vader : 소셜 미디어 텍스트를 분석할 때 사용. 대용량 텍스트에 비교적으로 잘 적용됨
  4. Pattern


### SentiWordNet
Synset = 단어.품사.서로 다른 의미의 갯수
ex) present.n.01 = present의 명사 의미 첫번째 >> 라고 해석하면 됨

* 어휘 간의 관계를 유사도로 나타낼 수 있음
--> path_similarity()

단어별 synset을 반복해 다른 단어의 synset과 유사도를 측정할 수 있음.

* 분석 과정
    1. 문서를 문장 단위로 분해
    2. 문장을 단어 단위로 토큰화 > 어근 추출 > 품사 태킹
    3. 품사 태킹된 단어 기반으로 synset 객체와 senti_synset 객체 생성
    4. senti_synset 객체에서 긍정/부정 감성 지수 구하고 모두 더해서 특정 임계치 이상일 때 긍정, 아니면 부정으로 판단


### Vader
SentimentIntensityAnalyzer 객체 생성
>> 문서 별로 polarity_scores() 메서드 호출 = 감성 점수 구하기
>> 감성 점수가 특정 값 이상이면 긍정, 아니면 부정 판단

counpund는 neg, neu, pos 점수를 조합해서 -1~1 사이의 감성 지수를 표현함
0.1 이상이면 긍정, 그 이하면 부정으로 판단함

In [2]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [5]:
from sklearn.datasets import fetch_20newsgroups

train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target


print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)

X_train_cnt_vect = cnt_vect.transform(X_train)

X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.617


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.678


In [9]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.690


In [11]:
from sklearn.model_selection import GridSearchCV

params={'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter: ', grid_cv_lr.best_params_)

pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter:  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.704


In [12]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
                     ('lr_clf', LogisticRegression(random_state=156))])

In [14]:
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
                     ('lr_clf', LogisticRegression(solver='liblinear', C=10))])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.704


In [ ]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

params={'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

grid_cv_pipe=GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred=grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7533152393023013
Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


In [18]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
review_df = pd.read_csv('/content/drive/MyDrive/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df.head(3)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [19]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [20]:
## replace를 이용해 HTML 태그를 제거, 숫자/특수문자도 공락으로 변경 by. 정규 표현식

import re

review_df['review'] = review_df['review'].str.replace('<br />', ' ')
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ",x))

In [21]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace = False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8861, ROC_AUC는 0.9503


In [26]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC_AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8936, ROC_AUC는 0.9598


In [28]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [29]:
from nltk.corpus import wordnet as wn

term='present'

synsets = wn.synsets(term)     ## 여러 개의 Synset 객체를 가지는 리스트
print('synsets() 반환 type : ', type(synsets))
print('synsets() 반환 값 개수 : ', len(synsets))
print('synsets() 반환 값 : ', synsets)

synsets() 반환 type :  <class 'list'>
synsets() 반환 값 개수 :  18
synsets() 반환 값 :  [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [30]:
for synset in synsets:
  print('##### Synset name :', synset.name(), '#####')
  print('POS :', synset.lexname())
  print('Definition: ', synset.definition())
  print('Lemmas :', synset.lemma_names())

##### Synset name : present.n.01 #####
POS : noun.time
Definition:  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas : ['present', 'nowadays']
##### Synset name : present.n.02 #####
POS : noun.possession
Definition:  something presented as a gift
Lemmas : ['present']
##### Synset name : present.n.03 #####
POS : noun.communication
Definition:  a verb tense that expresses actions or states at the time of speaking
Lemmas : ['present', 'present_tense']
##### Synset name : show.v.01 #####
POS : verb.perception
Definition:  give an exhibition of to an interested audience
Lemmas : ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name : present.v.02 #####
POS : verb.communication
Definition:  bring forward and present to the mind
Lemmas : ['present', 'represent', 'lay_out']
##### Synset name : stage.v.01 #####
POS : verb.creation
Definition:  perform (a play), especially on a stage
Lemmas : ['stage', 'present', 're

In [31]:
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split(',')[0] for entity in entities]

for entity in entities:
  similarity = [round(entity.path_similarity(compared_entity), 2)
                for compared_entity in entities]
  similarities.append(similarity)

similarity_df = pd.DataFrame(similarities, columns = entity_names, index=entity_names)
similarity_df

,tree.n.01,lion.n.01,tiger.n.02,cat.n.01,dog.n.01
tree.n.01,1.00,0.07,0.07,0.08,0.12
lion.n.01,0.07,1.00,0.33,0.25,0.17
tiger.n.02,0.07,0.33,1.00,0.25,0.17
cat.n.01,0.08,0.25,0.25,1.00,0.20
dog.n.01,0.12,0.17,0.17,0.20,1.00


In [32]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 개수 :', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 개수 : 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [34]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수:', father.pos_score())
print('father 긍정감성 지수:', father.neg_score())
print('father 긍정감성 지수:', father.obj_score())
print('\n')

fabulous = swn.senti_synset('fabulous.a.01')
print('father 긍정감성 지수:', father.pos_score())
print('father 긍정감성 지수:', father.neg_score())

father 긍정감성 지수: 0.0
father 긍정감성 지수: 0.0
father 긍정감성 지수: 1.0


father 긍정감성 지수: 0.0
father 긍정감성 지수: 0.0


In [35]:
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

In [38]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
  sentiment = 0.0
  tokens_count=0

  lemmatizer = WordNetLemmatizer()
  raw_sentences = sent_tokenize(text)

  for raw_sentence in raw_sentences:
    tagged_sentence = pos_tag(word_tokenize(raw_sentence))
    for word, tag in tagged_sentence:
      wn_tag = penn_to_wn(tag)
      if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        continue
      lemma = lemmatizer.lemmatize(word, pos=wn_tag)

      if not lemma:
        continue
      synsets = wn.synsets(lemma, pos = wn_tag)

      if not synsets:
        continue
      synset = synsets[0]
      swn_synset = swn.senti_synset(synset.name())
      sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
      tokens_count +=1

  if not tokens_count:
    return 0

  if sentiment >= 0:
    return 1

  return 0


In [39]:
review_df['preds'] = review_df['review'].apply( lambda x:swn_polarity(x))

y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [42]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print("정확도 : ", np.round(accuracy_score(y_target, preds), 4))
print("정밀도 : ", np.round(precision_score(y_target, preds), 4))
print("재현율 : ", np.round(recall_score(y_target, preds), 4))

[[7669 4831]
 [3635 8865]]
정확도 :  0.6614
정밀도 :  0.6473
재현율 :  0.7092


In [43]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


In [ ]:
def vader_polarity(review,threshold=0.1):
    analyzer=SentimentIntensityAnalyzer()
    scores=analyzer.polarity_scores(review)

    agg_score=scores['compound']
    final_sentiment=1 if agg_score>=threshold else 0
    return final_sentiment

review_df['vader_preds']=review_df['review'].apply(lambda x:vader_polarity(x, 0.1))
y_target=review_df['sentiment'].values
vader_preds=review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target, vader_preds), 4))
print("정밀도:", np.round(precision_score(y_target, vader_preds), 4))
print("재현율:", np.round(recall_score(y_target, vader_preds), 4))

[[ 6749  5751]
 [ 1856 10644]]
정확도: 0.6957
정밀도: 0.6492
재현율: 0.8515
